In [85]:
import boto3, json, math

class ToolError(Exception):
    pass


In [86]:
def call_bedrock(message_list, tool_list):
    session = boto3.Session()

    bedrock = session.client(service_name='bedrock-runtime')
    
    response = bedrock.converse(
        modelId="amazon.nova-pro-v1:0",
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0.2
        },
        toolConfig={ "tools": tool_list }
    )
    
    return response


In [87]:
def get_tool_result(tool_use_block):
    
    tool_use_name = tool_use_block['name']
            
    print(f"Using tool {tool_use_name}")
    
    # Note: We're deliberately excluding tangent so something magical can happen
    if tool_use_name == 'cosine':
        return math.cos(tool_use_block['input']['x'])
    elif tool_use_name == 'sine':
        return math.sin(tool_use_block['input']['x'])
    elif tool_use_name == 'divide_numbers':
        return tool_use_block['input']['x'] / tool_use_block['input']['y'] 
    else:
        raise ToolError(f"Invalid function name: {tool_use_name}")


In [88]:
def handle_response(response_message):
    
    response_content_blocks = response_message['content']
    
    follow_up_content_blocks = []
    
    for content_block in response_content_blocks:
        if 'toolUse' in content_block:
            tool_use_block = content_block['toolUse']
            
            try:
                tool_result_value = get_tool_result(tool_use_block)
                
                if tool_result_value is not None:
                    follow_up_content_blocks.append({
                        "toolResult": {
                            "toolUseId": tool_use_block['toolUseId'],
                            "content": [
                                { "json": { "result": tool_result_value } }
                            ]
                        }
                    })
                
            except ToolError as e:
                follow_up_content_blocks.append({ 
                    "toolResult": {
                        "toolUseId": tool_use_block['toolUseId'],
                        "content": [  { "text": repr(e) } ],
                        "status": "error"
                    }
                })
        
    
    if len(follow_up_content_blocks) > 0:
        
        follow_up_message = {
            "role": "user",
            "content": follow_up_content_blocks,
        }
        
        return follow_up_message
    else:
        return None


In [89]:
def run_loop(prompt, tool_list):
    MAX_LOOPS = 6
    loop_count = 0
    continue_loop = True
    
    message_list = [
        {
            "role": "user",
            "content": [ { "text": prompt } ]
        }
    ]
    
    while continue_loop:
        response = call_bedrock(message_list, tool_list)
        
        response_message = response['output']['message']
        message_list.append(response_message)
        
        loop_count = loop_count + 1
        
        if loop_count >= MAX_LOOPS:
            print(f"Hit loop limit: {loop_count}")
            break
        
        follow_up_message = handle_response(response_message)
        
        if follow_up_message is None:
            # No remaining work to do, return final response to user
            continue_loop = False 
        else:
            message_list.append(follow_up_message)
            
    return message_list


In [6]:
tools = [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function."
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "sine",
            "description": "Calculate the sine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function."
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "tangent",
            "description": "Calculate the tangent of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function."
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    },
    {
        "toolSpec": {
            "name": "divide_numbers",
            "description": "Divide x by y.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The numerator."
                        },
                        "y": {
                            "type": "number",
                            "description": "The denominator."
                        }
                    },
                    "required": ["x", "y"]
                }
            }
        }
    }
]


In [7]:
messages = run_loop("What is the tangent of 7?", tools)

print("\nMESSAGES:\n")
print(json.dumps(messages, indent=4))


Using tool tangent

MESSAGES:

[
    {
        "role": "user",
        "content": [
            {
                "text": "What is the tangent of 7?"
            }
        ]
    },
    {
        "role": "assistant",
        "content": [
            {
                "text": "<thinking> To find the tangent of 7, I need to use the tangent tool with the argument 7. </thinking>\n"
            },
            {
                "toolUse": {
                    "toolUseId": "tooluse_P8-yrLWaQO2YtudCmeV9oQ",
                    "name": "tangent",
                    "input": {
                        "x": 7
                    }
                }
            }
        ]
    },
    {
        "role": "user",
        "content": [
            {
                "toolResult": {
                    "toolUseId": "tooluse_P8-yrLWaQO2YtudCmeV9oQ",
                    "content": [
                        {
                            "text": "ToolError('Invalid function name: tangent')"
                  

In [36]:

tool_list =[
    {
        "toolSpec": {
            "name": "identify_complaints_filters",
            "description": """Your job is to first look at the query's query and determine if it requires filtering the complaints first based off the following columns 
            client_name - name of the company making the complaint, you will need to use the like filter since users might not type if the exact name
            client_region - possible values ['MC','LC','ICB'] where MC= mid-corporate, LC = Large corporate, ICB = International corprorate
            complaint_date - date of the complaint

            Note: if you're not sure then don't output anything. This is used for metadata filter for the vectorstore. You will need to use the following
            $eq (equals)
            $neq (not equals)
            $gt (greater than)
            $lt (less than)
            $gte (greater than or equal)
            $lte (less than or equal)
            $in (membership in list)
            $nin (not in list)
            $and (all conditions must match)
            $or (any condition must match)
            $not (negation of condition)

            Example1: user_query: Show me all complaints in region MC
            output: = {"client_region": {"$eq": "MC"}}

            Example2: user_query: Show me all complaints in region LC and after 15th June 2024
            output: = {"client_region": {"$eq": "MC"},"client_region": {"$gt": "2024-06-15"}}

            
            """,
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "array",
                            "description": "output filter as a dict"
                        }
                    },
                    "required": ["x"]
                }
            }
        }}


In [54]:
query_example = "is there any complaints relating to MC"
message_list = [
        {
            "role": "user",
            "content": [ { "text": query_example } ]
        }
    ]

In [55]:
import boto3
session = boto3.Session()

bedrock = session.client(service_name='bedrock-runtime')

response = bedrock.converse(
    modelId="amazon.nova-pro-v1:0",
    messages=message_list,
    inferenceConfig={
        "maxTokens": 2000,
        "temperature": 0.2
    },
    toolConfig={ "tools": tool_list }
)

{'ResponseMetadata': {'RequestId': 'be67d74e-b33a-4353-b426-09a8aec91b3e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Sat, 18 Jan 2025 14:51:32 GMT',
   'content-type': 'application/json',
   'content-length': '504',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'be67d74e-b33a-4353-b426-09a8aec91b3e'},
  'RetryAttempts': 0},
 'output': {'message': {'role': 'assistant',
   'content': [{'text': "<thinking> The user is asking about complaints relating to 'MC', which likely refers to the 'client_region'. I need to filter the complaints based on this region. </thinking>\n"},
    {'toolUse': {'toolUseId': 'tooluse_RZ8-NBvLTVqT_ZuLX7B_Mg',
      'name': 'identify_complaints_filters',
      'input': {'x': [{'client_region': {'$eq': 'MC'}}]}}}]}},
 'stopReason': 'tool_use',
 'usage': {'inputTokens': 849, 'outputTokens': 133, 'totalTokens': 982},
 'metrics': {'latencyMs': 1105}}

In [81]:
response['output']['message']['content'][1]

{'toolUse': {'toolUseId': 'tooluse_RZ8-NBvLTVqT_ZuLX7B_Mg',
  'name': 'identify_complaints_filters',
  'input': {'x': [{'client_region': {'$eq': 'MC'}}]}}}

In [90]:
handle_response(response['output']['message'])

Using tool identify_complaints_filters


{'role': 'user',
 'content': [{'toolResult': {'toolUseId': 'tooluse_RZ8-NBvLTVqT_ZuLX7B_Mg',
    'content': [{'text': "ToolError('Invalid function name: identify_complaints_filters')"}],
    'status': 'error'}}]}

In [57]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

In [58]:
from langchain.embeddings import BedrockEmbeddings
import boto3
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name='us-east-1',
)
embeddings = BedrockEmbeddings(
            client=bedrock_client, 
            model_id="amazon.titan-embed-text-v2:0"
        )


In [59]:
vector_store = FAISS.load_local('complaints.vs',embeddings,allow_dangerous_deserialization=True)

In [60]:
retriever = vector_store.as_retriever(search_kwargs={'filter':{'client_name': {'$like': 'Iportal'}}} )

In [61]:
docs = retriever.invoke('bring me back all the iportal complaints')

In [62]:
docs

[]